## This is for testing the WebScraping class

In [1]:
import requests, random, Universities
from urllib.parse import urlparse
from UserAgents import UserAgents
from User import User
from bs4 import BeautifulSoup

class WebScraping:
    bs4 = None
    def __init__(self):
        self.linkFilterPrefixes = ["/search", "q=", "/?", "/advanced_search"]
        self.linkFilterSearches = ["google", "facebook", "instagram", "linkedin", "twitter", "ratemyprofessors",
                                   "coursicle"]
        bs4 = BeautifulSoup()

    '''
    The initial search perform a google search on the user using the query "{FIRST LAST} {INSTITUTION}"
    and returns all the links
    '''
    def initial_search(self, user: User):
        # user_name = "+".join(user.research_data['name']) + f" {user.research_data['institution']}"

        # # Tweaks should be done here, maybe make a link scraper by itself

        # #search_url = f"https://www.google.com/search?q=%22{user_name}%22"
        # search_url = f"https://www.google.com/search?q={user_name}"

        # # End of tweaks
        # req = self.request(search_url)
        # bs = BeautifulSoup(req.content, 'html.parser')

        # # Select every single <a> element
        # raw_links = bs.select("a")
        # # Filter links that do not contain "google.com" or start with the prefixes defined.
        # links = [link['href'] for link in raw_links if not any(link['href'].startswith(prefix) 
        #             or link['href'].find('google.com') > 0 for prefix in self.linkFilterPrefixes)] 
        
        # # Filter links that don't contain searches
        # links = [link for link in links if not any(link.find(search) > -1 for search in self.linkFilterSearches)]

        # # Only grab the relevent part of the link if it includes more in it
        # links = [link.split("/url?q=")[-1].split("&sa")[0] for link in links]
        # user.initial_search_links = links
        # return links


        user_name = "+".join(user.research_data['name']) + f" {user.research_data['institution']}"
        links = []

        for page_number in range(3):
            start_index = page_number * 10
            search_url = f"https://www.google.com/search?q={user_name}&start={start_index}"

            req = self.request(search_url)
            bs = BeautifulSoup(req.content, 'html.parser')

            # Select every single <a> element
            raw_links = bs.select("a")
            # Filter links that do not contain "google.com" or start with the prefixes defined.
            filtered_links = [link['href'] for link in raw_links if not any(link['href'].startswith(prefix) 
                        or link['href'].find('google.com') > 0 for prefix in self.linkFilterPrefixes)] 
        
            # Filter links that don't contain searches
            filtered_links = [link for link in filtered_links if not any(link.find(search) > -1 for search in self.linkFilterSearches)]

            # Only grab the relevent part of the link if it includes more in it
            links += [link.split("/url?q=")[-1].split("&sa")[0] for link in filtered_links]

        user.initial_search_links = links
        # links = list(set(links))
        return links
    
    '''
    Verify if the link is relevent to the researcher. 2/3 is required to be used.
    1. First checks if the institution can be found on the page text.
    2. Checks if the researchers name can be found on the page check.
    3. Check if the URL is from their institution.
    '''
    def verify_link_relevancy(self, link: str, page_data: str, user: User):
        page_data = page_data.lower()
        user_name = "+".join(user.research_data['name']).lower()
        checks = 0
        reason = ""

        # Check 1
        if page_data.find(user.research_data['institution'].lower()):
            checks += 1
            reason += "Institution found | "

        # Check 2
        if page_data.find(user_name):
            checks += 1
            reason += "Researcher name found | "
        
        # Check 3
        if Universities.findUniversityLink(user.research_data['institution']).find(link) > -1:
            checks += 1
            reason += "University website verified"
        return (checks >= 2, checks, reason)
        
    ''' 
    Scrape the webpage and get the webtext without HTML tags
    then check verify the source is reputable by a 3 part check method
    '''
    def scrape_webpage(self, link: str, user: User):
        # Request the page and convert to BS4
        req = self.request(link)
        bs = BeautifulSoup(req.content, 'html.parser')
        
        # Grab only the webtext (text without HTML tags)
        webtext = bs.get_text().replace('\n', '').replace('"', '').strip()
        
        # Parse the URL so that we can only get the base domain
        parsed_url = urlparse(link)
        domain_parts = parsed_url.netloc.split('.')
        domain = '.'.join(domain_parts[-2:])

        # Do a 3 part check on the domain, webtext, and the user to verify it pertains to the user
        verified, check, reason = self.verify_link_relevancy(domain, webtext, user)

        # <TODO>
        #NOT DONE
        print(verified, check, reason)
        return webtext
        
    
    '''
    Internal request method that faciliates parameters and headers
    :return: `Response`
    '''
    def request(self, link) -> requests.Response:
        return requests.get(link, self.genHeaders())

    '''
    Generate new headers
    '''
    def genHeaders(self) -> dict:
        return {
        'User-agent': self.getRandAgent()
        }

    ''' 
    Returns a random UserAgent for the headers
    '''
    def getRandAgent(self) -> str:
        return UserAgents[random.randrange(len(UserAgents))]


<center><b>Test with researcher Zheng Xiang from VT</b></center>

In [4]:
ws = WebScraping()
zheng = User("Zheng Xiang", "Virginia Tech")
ws.initial_search(zheng)
zheng.initial_search_links

ws.scrape_webpage(zheng.initial_search_links[0], zheng)



True 3 Institution found | Researcher name found | University website verified


'Zheng (Phil) Xiang  | Howard Feiertag Department of Hospitality & Tourism Management | Virginia TechSkip to main contentSkip to searchVirginia Tech®homeUniversal Access ToggleUniversal AccessClose Universal Access dialogUniversal Access OptionsReport a barrierAccessibility portalPause all background videosUnderline all links   ApplyVisitGiveShopHokie GearApparel, clothing, gear and merchandiseHokie ShopUniversity Bookstore, merchandise and giftsHokie License PlatesPart of every Virginia Tech plate purchase funds scholarshipsResources forFuture StudentsCurrent StudentsParents and FamiliesFaculty and StaffAlumniIndustry and Partners  Howard Feiertag Department of Hospitality & Tourism ManagementMenuPamplin College of Business\xa0\xa0/Howard Feiertag Department of Hospitality & Tourism ManagementAbout DirectoryUndergraduateUndergraduate Submenu ToggleRequirementsCoursesField StudyStudy AbroadStudent OrganizationsScholarshipsReal Estate Double MajorFrequently Asked QuestionsEvent and Expe

In [7]:
ws.initial_search(zheng)


['https://engineering.virginia.edu/faculty/sean-r-agnew',
 'https://publicsearch.people.virginia.edu/person/sra4p',
 'https://3c.virginia.edu/people/sra4p',
 'https://www.sciencedirect.com/author/57203775288/sean-r-agnew',
 'https://www.researchgate.net/profile/Sr-Agnew',
 'https://www.apollo.io/people/Sean/Agnew/54fb77c87468692f6b40cc0c',
 'https://www.apollo.io/people/Sean/Agnew/54fb77c87468692f6b40cc0c',
 'https://www.jobs.virginia.gov/jobs/research-associate-in-materials-science-and-engineering-charlottesville-virginia-united-states-7f542dfc-3cac-4596-b156-0fd277748371',
 'https://link.springer.com/article/10.1007/s11837-006-0026-8',
 'https://research.com/u/bjorn-clausen',
 'https://www.tms.org/communities/ftattachments/jom-0405-20.pdf',
 'https://virginianuclear.org/about-vnec/leadership/',
 'https://lansce.lanl.gov/users/rosen-scholar/past-scholars.php',
 'http://hxdata.chisa.edu.cn/professor/detail/id/5210',
 'https://www.journals.elsevier.com/international-journal-of-plasticit

In [6]:
sean = User("Sean R. Agnew", "University of Virginia")

ws.initial_search(sean)
sean.initial_search_links


['https://engineering.virginia.edu/faculty/sean-r-agnew',
 'https://publicsearch.people.virginia.edu/person/sra4p',
 'https://3c.virginia.edu/people/sra4p',
 'https://www.sciencedirect.com/author/57203775288/sean-r-agnew',
 'https://www.researchgate.net/profile/Sr-Agnew',
 'https://www.apollo.io/people/Sean/Agnew/54fb77c87468692f6b40cc0c',
 'https://www.apollo.io/people/Sean/Agnew/54fb77c87468692f6b40cc0c',
 'https://www.tms.org/communities/ftattachments/jom-0405-20.pdf',
 'https://virginianuclear.org/about-vnec/leadership/',
 'https://research.com/u/bjorn-clausen',
 'https://lansce.lanl.gov/users/rosen-scholar/past-scholars.php',
 'http://hxdata.chisa.edu.cn/professor/detail/id/5210',
 'https://www.journals.elsevier.com/international-journal-of-plasticity/editorial-board',
 'https://link.springer.com/chapter/10.1007/978-3-319-48114-2_38',
 'https://www.cambridge.org/core/services/aop-cambridge-core/content/view/984A4D95655F12058458A249DCC076CC/S1431927610093992a.pdf/simple_algorithm_t

In [58]:
ws.scrape_webpage(zheng.initial_search_links[15], zheng)

True 2 Institution found | Researcher name found | 


''

In [59]:
zheng.initial_search_links[15]

'https://xiangcheng.link/'

In [14]:
from urllib.parse import urlparse

url = 'https://htm.pamplin.vt.edu/directory/xiang.html&sa=U&ved=2ahUKEwj7sYqd_Nf9AhUkFVkFHXX6Bw8QFnoECAkQAg&usg=AOvVaw2sC1rEKB_bDYW5IiSR_gu4'
parsed_url = urlparse(url)
print(parsed_url)
domain_parts = parsed_url.netloc.split('.')
domain = '.'.join(domain_parts[-2:])
domain

ParseResult(scheme='https', netloc='htm.pamplin.vt.edu', path='/directory/xiang.html&sa=U&ved=2ahUKEwj7sYqd_Nf9AhUkFVkFHXX6Bw8QFnoECAkQAg&usg=AOvVaw2sC1rEKB_bDYW5IiSR_gu4', params='', query='', fragment='')


'vt.edu'